In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.api import OLS, add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
import yfinance as yf
import requests
import os

warnings.filterwarnings('ignore')

api_key = '8481ed700f2bf3bb0575f4e9d88f8bbf'             # Financial modeling prep API Key

sys.path.append('FinRecipes/')
from downloader import fmp_download_constituents, yf_fetch_industry_info

df = pd.read_excel('./data/sp500-index-components-05-21-2024.xlsx')
df = df.head(500)

In [10]:


# # Apply the function to all symbols
# df[['sector', 'industry']] = df['Symbol'].apply(lambda x: yf_fetch_industry_info(x)).apply(pd.Series)
# df.dropna(subset=['industry'], inplace=True)


# def fetch_all_historical_data(symbol, api_key):
#     data = {}

#     # Fetch ratios
#     response = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{symbol}?period=quarter&apikey={api_key}')
#     if response.status_code == 200:
#         data['ratios'] = response.json()
#     else:
#         print(f"No ratios data available for ticker: {symbol}")

#     # Fetch key metrics
#     response = requests.get(f'https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?period=quarter&apikey={api_key}')
#     if response.status_code == 200:
#         data['key_metrics'] = response.json()
#     else:
#         print(f"No key metrics data available for ticker: {symbol}")

#     # Fetch income statement
#     response = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=quarter&apikey={api_key}')
#     if response.status_code == 200:
#         data['income_statement'] = response.json()
#     else:
#         print(f"No income statement data available for ticker: {symbol}")

#     # Fetch balance sheet
#     response = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?period=quarter&apikey={api_key}')
#     if response.status_code == 200:
#         data['balance_sheet'] = response.json()
#     else:
#         print(f"No balance sheet data available for ticker: {symbol}")

#     # Fetch financial growth
#     response = requests.get(f'https://financialmodelingprep.com/api/v3/financial-growth/{symbol}?period=quarter&apikey={api_key}')
#     if response.status_code == 200:
#         data['financial_growth'] = response.json()
#     else:
#         print(f"No financial growth data available for ticker: {symbol}")

#     return data

# # extract relevant metrics
# def extract_metrics(data):
#     metrics = []

#     if data.get('ratios'):
#         for ratio in data['ratios']:
#             metrics.append({
#                 'date': ratio['date'],
#                 'priceToBookRatio': ratio.get('priceToBookRatio'),
#                 'priceToSalesRatio': ratio.get('priceToSalesRatio'),
#                 'priceEarningsRatio': ratio.get('priceEarningsRatio'),
#                 'netProfitMargin': ratio.get('netProfitMargin'),
#                 'returnOnAssets': ratio.get('returnOnAssets'),
#                 'returnOnEquity': ratio.get('returnOnEquity'),
#                 'returnOnCapitalEmployed': ratio.get('returnOnCapitalEmployed'),
#                 'operatingProfitMargin': ratio.get('operatingProfitMargin'),
#                 'currentRatio': ratio.get('currentRatio'),
#                 'quickRatio': ratio.get('quickRatio'),
#                 'cashRatio': ratio.get('cashRatio'),
#                 'assetTurnover': ratio.get('assetTurnover'),
#                 'inventoryTurnover': ratio.get('inventoryTurnover'),
#                 'receivablesTurnover': ratio.get('receivablesTurnover'),
#                 'payablesTurnover': ratio.get('payablesTurnover'),
#                 'cashConversionCycle': ratio.get('cashConversionCycle'),
#                 'dividendPayoutRatio': ratio.get('dividendPayoutRatio'),
#                 'earningsYield': ratio.get('earningsYield'),
#                 'dividendYield': ratio.get('dividendYield'),
#                 'priceToFreeCashFlowsRatio': ratio.get('priceToFreeCashFlowsRatio')
#             })

#     if data.get('key_metrics'):
#         for key_metric in data['key_metrics']:
#             matching_date = next((item for item in metrics if item['date'] == key_metric['date']), None)
#             if matching_date:
#                 matching_date.update({
#                     'enterpriseValueOverEBITDA': key_metric.get('enterpriseValueOverEBITDA'),
#                     'freeCashFlowYield': key_metric.get('freeCashFlowYield'),
#                     'debtToEquity': key_metric.get('debtToEquity'),
#                     'debtToAssets': key_metric.get('debtToAssets'),
#                     'netDebtToEBITDA': key_metric.get('netDebtToEBITDA'),
#                     'interestCoverage': key_metric.get('interestCoverage'),
#                     'workingCapital': key_metric.get('workingCapital')
#                 })

#     if data.get('income_statement'):
#         for income in data['income_statement']:
#             matching_date = next((item for item in metrics if item['date'] == income['date']), None)
#             if matching_date:
#                 matching_date.update({
#                     'eps': income.get('eps'),
#                     'incomeBeforeTax': income.get('incomeBeforeTax')
#                 })

#     if data.get('balance_sheet'):
#         for balance in data['balance_sheet']:
#             matching_date = next((item for item in metrics if item['date'] == balance['date']), None)
#             if matching_date:
#                 matching_date.update({
#                     'totalAssets': balance.get('totalAssets')
#                 })

#     if data.get('financial_growth'):
#         for growth in data['financial_growth']:
#             matching_date = next((item for item in metrics if item['date'] == growth['date']), None)
#             if matching_date:
#                 matching_date.update({
#                     'revenueGrowth': growth.get('revenueGrowth'),
#                     'epsgrowth': growth.get('epsgrowth'),
#                     'operatingIncomeGrowth': growth.get('operatingIncomeGrowth'),
#                     'freeCashFlowGrowth': growth.get('freeCashFlowGrowth'),
#                     'assetGrowth': growth.get('assetGrowth')
#                 })

#     # economic rate of return (er)
#     for metric in metrics:
#         if ('incomeBeforeTax' in metric and 'totalAssets' in metric and
#             metric['totalAssets'] is not None and metric['totalAssets'] != 0):
#             metric['er'] = metric['incomeBeforeTax'] / metric['totalAssets']
#         else:
#             metric['er'] = None

#     return pd.DataFrame(metrics)

In [11]:
# industry_dfs = {}

# # Fetching and appending the historical data
# for idx, row in df.iterrows():
#     symbol = row['Symbol']
#     industry = row['industry']
#     financial_data = fetch_all_historical_data(symbol, api_key)
#     metrics_df = extract_metrics(financial_data)
#     metrics_df['Symbol'] = symbol
#     metrics_df['industry'] = industry


#     if industry in industry_dfs:
#         industry_dfs[industry] = pd.concat([industry_dfs[industry], metrics_df], ignore_index=True)
#     else:
#         industry_dfs[industry] = metrics_df

In [12]:
# output_dir = './data/industry_data/'
# os.makedirs(output_dir, exist_ok=True)

# for industry, df in industry_dfs.items():
#     filename = os.path.join(output_dir, f"{industry.replace(' ', '_')}.xlsx")
#     df.to_excel(filename, index=False)
#     print(f"Saved data for industry {industry} to {filename}")

In [13]:
def perform_regression(df, industry):
    print(f"Performing regression for industry: {industry}")
    print(f"Initial shape of DataFrame: {df.shape}")

    # Drop columns that are entirely NaN
    industry_df = df.drop(['Symbol', 'industry', 'date'], axis=1).dropna(axis=1, how='all')
    print(f"Shape after dropping columns with NaNs: {industry_df.shape}")

    # degree of freedom
    if industry_df.shape[0] <= industry_df.shape[1]:
        print(f"Not enough data to perform regression for industry: {industry}")
        return None

    # # Imputation
    # knn_imputer = KNNImputer(n_neighbors=5)
    # industry_df_imputed = pd.DataFrame(knn_imputer.fit_transform(industry_df), columns=industry_df.columns)
    # print(f"Shape after imputing rows: {industry_df_imputed.shape}")

    industry_df_imputed = industry_df.fillna(method='bfill').fillna(method='ffill')

    # Standardize the features
    scaler = StandardScaler()
    X = industry_df_imputed.drop('er', axis=1)
    X_scaled = scaler.fit_transform(X)
    y = industry_df_imputed['er']


    # VIF
    X_vif = pd.DataFrame(X_scaled, columns=X.columns)
    vif_data = pd.DataFrame()
    vif_data['feature'] = X_vif.columns
    vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]


    # Removing features with high VIF
    high_vif_features = vif_data[vif_data['VIF'] > 10]['feature'].tolist()
    X_vif.drop(columns=high_vif_features, inplace=True)
    print(f"Features with high VIF removed: {high_vif_features}")
    print(f"Shape after removing high VIF features: {X_vif.shape}")

    # Aligning indices
    X_vif.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    # Adding constant for OLS
    X_vif = add_constant(X_vif)

    # Performing backward elimination
    def backward_elimination(data, target, significance_level=0.05):
        features = data.columns.tolist()
        while len(features) > 0:
            X = data[features]
            X = add_constant(X)
            model = OLS(target, X).fit()
            p_values = model.pvalues[1:]  # exclude the intercept
            max_p_value = p_values.max()
            if max_p_value > significance_level:
                excluded_feature = p_values.idxmax()
                features.remove(excluded_feature)
            else:
                break
        print(features)

        with open(industry+".txt", "w") as output:
            output.write(str(features))

        # file = open("industry"+".txt", 'w') 
        # file.write(features) 
        # file.close() 

        return model

    # Performing backward elimination and using OLS
    final_model = backward_elimination(X_vif, y)
    print(f"Summary for {industry}")
    print(final_model.summary())

    # Extracting OLS coefficients
    ols_coefficients = final_model.params



    # Remove constant for Ridge and Lasso
    X_vif = X_vif.drop(columns=['const'])

    # Ridge Regression
    ridge = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=5)
    ridge.fit(X_vif, y)
    print(f"Ridge Regression Best Alpha for {industry}:", ridge.alpha_)
    print(f"Ridge Regression Coefficients for {industry}:", ridge.coef_)



    # Lasso Regression
    lasso = LassoCV(alphas=np.logspace(-6, 6, 13), cv=5)
    lasso.fit(X_vif, y)
    print(f"Lasso Regression Best Alpha for {industry}:", lasso.alpha_)
    print(f"Lasso Regression Coefficients for {industry}:", lasso.coef_)



    # Cross-validation to evaluate the model
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    ridge_cv_scores = cross_val_score(ridge, X_vif, y, cv=kf, scoring='r2')
    lasso_cv_scores = cross_val_score(lasso, X_vif, y, cv=kf, scoring='r2')

    print(f"Ridge Regression CV Scores for {industry}:", ridge_cv_scores)
    print(f"Ridge Regression CV Mean Score for {industry}:", np.mean(ridge_cv_scores))
    print(f"Lasso Regression CV Scores for {industry}:", lasso_cv_scores)
    print(f"Lasso Regression CV Mean Score for {industry}:", np.mean(lasso_cv_scores))

    return {
        'ridge': {
            'alpha': ridge.alpha_,
            'coefficients': pd.Series(ridge.coef_, index=X_vif.columns)
        },
        'lasso': {
            'alpha': lasso.alpha_,
            'coefficients': pd.Series(lasso.coef_, index=X_vif.columns)
        },
        'ols': {
            'coefficients': ols_coefficients
        },
        'final_model_summary': final_model.summary()
    }

In [14]:
industry_data_path = './data/industry_data/'

# List all files in the directory
industry_files = os.listdir(industry_data_path)
industry_dfs = {}

# Load each file into a DataFrame
for file in industry_files:
    if file.endswith('.xlsx'):
        industry_name = file.split('.')[0]
        file_path = os.path.join(industry_data_path, file)
        industry_dfs[industry_name] = pd.read_excel(file_path)

In [15]:
# DataFrames for storing coefficients
ridge_coeffs_df = pd.DataFrame()
lasso_coeffs_df = pd.DataFrame()
ols_coeffs_df = pd.DataFrame()

# Running regression for each industry
results = {}
for industry, df in industry_dfs.items():
    print(f"Industry: {industry}, Number of Symbols: {len(df['Symbol'].unique())}, Data Shape: {df.shape}")
    if len(df['Symbol'].unique()) > 2 and df.shape[0] > df.shape[1] - 5:
        result = perform_regression(df, industry)
        if result is not None:
            results[industry] = result
            ridge_coeffs_df[industry] = result['ridge']['coefficients']
            lasso_coeffs_df[industry] = result['lasso']['coefficients']
            ols_coeffs_df[industry] = result['ols']['coefficients']
    else:
        print(f"Skipping industry: {industry} due to insufficient data.")


Industry: Advertising_Agencies, Number of Symbols: 2, Data Shape: (310, 39)
Skipping industry: Advertising_Agencies due to insufficient data.
Industry: Aerospace_&_Defense, Number of Symbols: 12, Data Shape: (1516, 39)
Performing regression for industry: Aerospace_&_Defense
Initial shape of DataFrame: (1516, 39)
Shape after dropping columns with NaNs: (1516, 35)
Features with high VIF removed: ['priceToBookRatio', 'priceToSalesRatio', 'priceEarningsRatio', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'cashConversionCycle', 'netDebtToEBITDA']
Shape after removing high VIF features: (1516, 25)
['const', 'netProfitMargin', 'returnOnAssets', 'operatingProfitMargin', 'payablesTurnover', 'interestCoverage', 'eps', 'incomeBeforeTax', 'totalAssets', 'revenueGrowth', 'epsgrowth']
Summary for Aerospace_&_Defense
                            OLS Regression Results                            
Dep. Variable:                     er   R-squared:                       0.832
Model:      

In [16]:
# Save coefficients to CSV files
ridge_coeffs_df.index.name = 'Metrics' # Column header
lasso_coeffs_df.index.name = 'Metrics'
ols_coeffs_df.index.name = 'Metrics'

ridge_coeffs_df.to_csv('ridge_coefficients.csv')
lasso_coeffs_df.to_csv('lasso_coefficients.csv')
ols_coeffs_df.to_csv('ols_coefficients.csv')


for industry, result in results.items():
    print(f"Industry: {industry}")
    print("Ridge Regression Coefficients:", result['ridge']['coefficients'])
    print("Lasso Regression Coefficients:", result['lasso']['coefficients'])
    print(result['final_model_summary'])
    print("\n")

Industry: Aerospace_&_Defense
Ridge Regression Coefficients: netProfitMargin             -0.003530
returnOnAssets               0.021449
returnOnCapitalEmployed      0.000941
operatingProfitMargin        0.000398
assetTurnover                0.001075
inventoryTurnover           -0.000441
receivablesTurnover         -0.000472
payablesTurnover             0.001160
dividendPayoutRatio          0.000363
dividendYield               -0.000233
priceToFreeCashFlowsRatio    0.000298
enterpriseValueOverEBITDA    0.000173
freeCashFlowYield           -0.000118
debtToEquity                -0.000401
debtToAssets                 0.000379
interestCoverage             0.000918
workingCapital              -0.000071
eps                          0.002704
incomeBeforeTax              0.002752
totalAssets                 -0.001792
revenueGrowth                0.001918
epsgrowth                   -0.003499
operatingIncomeGrowth        0.000314
freeCashFlowGrowth           0.000088
assetGrowth                